In [1]:
from datetime import datetime, timedelta
import pandas as pd 
import datetime as dt
import fnmatch
import os
import pymysql
from sqlalchemy import create_engine
import sys
import json
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

# Obtener la fecha de hoy
hoy = datetime.now()

# Calcular la fecha de ayer
ayer = hoy - timedelta(days=1)

# Formatear la fecha como "aaaa-mm-dd"
fecha_ayer_formato = ayer.strftime("%Y-%m-%d")

fecha_ayer_formato


'2024-05-15'

In [2]:
cnx = pymysql.connect(host='159.89.90.197', port=3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')

In [3]:
path = f'./JERS/data/{fecha_ayer_formato}/'
if not os.path.exists(path): 
    os.makedirs(path) 

# DEU

In [4]:
cursor = cnx.cursor()
sql_query = f"""
            SELECT 
ids.id_distribuidor as `Clave Referencia`,
if(ids.tipo_prestamo = 'PRESTAMO PERSONAL RED', 'RED', cd.tipo_distribuidor) as 'tipo distribuidor',
"6" as `Tipo Operación`,
ids.id_red as `Contratante`,
IFNULL(dd.apellido_1,"X") as `Paterno Deudor`,
IFNULL(dd.apellido_2,"X") as `Materno Contratante`,
CONCAT(ifnull(dd.nombre_1,"")," ",ifnull(dd.nombre_2,"")) as `Nombre Contratante`,
"PF" as `Tipo de Persona`,
ifnull(dd.rfc,"X") as `RFC`,
CONCAT(ifnull(dd.calle_y_numero,"")," ",ifnull(dd.colonia,"")) as `Dirección`,
IFNULL(dd.delegacion,"X") as `Ciudad`,
IFNULL(dd.cp,"X") as `CP`,
IFNULL(dd.estado,"X") as `Estado`,
"N/A" as `Email`,
IFNULL(dd.telefono,"X") as `Teléfono`,
date(cd.fecha_aut_linea) as `Fecha Firma Contrato`,
if(cd.tipo_distribuidor = 'RED',ids.fin_credito,'2099-12-31') as `Fecha Vencimiento Contrato`,
if(cd.tipo_distribuidor = 'RED','16','0') as `Plazo`,
ids.id_red as `Crédito`,
"N" as `Anexo`,
cd.linea_de_credito  as `Monto Contrato`,
if(cd.tipo_distribuidor = 'RED','Semanal','Quincenal') as `Frecuencia de Pago`,
cd.proximo_pago_total_pago as `Pago Periódico`,
cd.linea_de_credito as `Monto Pagare`,
if(cd.tipo_distribuidor = 'RED',ids.tasa,"") AS `Tasa`,
dd.fecha_nacimiento as `Fecha de nacimiento contratante`,
IFNULL(dd.sexo,"X") as `Sexo`,
IFNULL(dd.curp,"X") as `CURP`
FROM cartera_distribuidor cd
LEFT JOIN (
		select 
			ca.id_distribuidor, 
            max(ca.id_red) as 'id_red', 
            max(ca.tipo_prestamo) as 'tipo_prestamo',
            max(tasa) as 'tasa',
            max(fin_credito) as 'fin_credito'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo,
                tasa,
                fin_credito
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) ids on cd.id_distribuidor = ids.id_distribuidor
LEFT JOIN demograficos_distribuidores dd on ids.id_distribuidor = dd.id
WHERE fecha = '{fecha_ayer_formato}' and 
ids.id_distribuidor IS NOT NULL and
dd.nombre_1 IS NOT NULL;
            """
cursor.execute(sql_query)
DEU = cursor.fetchall()

In [5]:
DEU = pd.DataFrame(DEU,columns = ['Clave Referencia', 'tipo distribuidor', 'Tipo Operación',
       'Contratante', 'Paterno Deudor', 'Materno Contratante',
       'Nombre Contratante', 'Tipo de Persona', 'RFC', 'Dirección', 'Ciudad',
       'CP', 'Estado', 'Email', 'Teléfono', 'Fecha Firma Contrato',
       'Fecha Vencimiento Contrato', 'Plazo', 'Crédito', 'Anexo',
       'Monto Contrato', 'Frecuencia de Pago', 'Pago Periódico',
       'Monto Pagare', 'Tasa', 'Fecha de nacimiento contratante', 'Sexo',
       'CURP'])

## Adecuaciones

In [6]:
DEU_id = np.array(DEU['Contratante'].unique(), dtype = int)
DEU['Fecha Firma Contrato'] = pd.to_datetime(DEU['Fecha Firma Contrato'])
print(DEU.shape)
for name in DEU.select_dtypes(include=['object']):
    DEU[name] = DEU[name].str.replace("|", " ")
print("Hay que revisar DEU" if (DEU.isna().any(0).sum() > 2) else "Todo bien con DEU")
DEU.head(1)

(10909, 28)
Todo bien con DEU


,Clave Referencia,tipo distribuidor,Tipo Operación,Contratante,Paterno Deudor,Materno Contratante,Nombre Contratante,Tipo de Persona,RFC,Dirección,...,Crédito,Anexo,Monto Contrato,Frecuencia de Pago,Pago Periódico,Monto Pagare,Tasa,Fecha de nacimiento contratante,Sexo,CURP
0,1,CLUB,6,1,DIAZ,FLORES,MARIA DE LOS ANGELES,PF,DIFA770122,SAN BERNARDINO no ext.14 Cofradia de la Luz,...,1,N,62000,Quincenal,0.0,62000,,1977-01-22,FEMENINO,DIFA770102MJCZLN08


In [7]:
if not os.path.exists("path/to/demo_folder"): 
    os.makedirs("path/to/demo_folder") 

In [8]:
DEU.to_csv(f'{path}/DEU_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

# MOVA

In [9]:
cursor = cnx.cursor()
sql_query = f"""
select 
    id_distribuidor as `Contrato`,
    "N" as `Anexo`,
    -- if(tipo_distribuidor = 'RED', id_distribuidor_linea_de_credito, id_distribuidor) as `Número de documento`,
    id_distribuidor as `Número de documento`,
    date(if(tipo_distribuidor = 'RED',fin_credito,'2099-12-31')) as `Fecha Vencimiento`,
    ROUND(sum(capital), 2) as `Monto Capital`,
    ROUND(sum(interes), 2) as `Monto intérés`,
    ROUND(sum(iva), 2) as `Monto IVA`,
    "FAC" AS `Tipo de Movimiento`
FROM
    (SELECT 
    cd.id_distribuidor,
    "N" as `Anexo`,
    cd.tipo_distribuidor,
    ca.capital,
    ca.interes,
    ca.iva,
    cd.fecha,
    ca.fin_credito,
    ca.no_pagos,
    ca.id_distribuidor_linea_de_credito
    FROM cartera_distribuidor cd
    LEFT JOIN colocacion_asociado ca on cd.id_distribuidor = ca.id_distribuidor
    WHERE fecha = '{fecha_ayer_formato}' and ca.tipo_prestamo = 'PRESTAMO PERSONAL RED') as temp

GROUP BY temp.id_distribuidor, fecha, fin_credito, tipo_distribuidor-- , temp.id_distribuidor_linea_de_credito
            """
cursor.execute(sql_query)
MOVA = cursor.fetchall()

In [10]:
MOVA = pd.DataFrame(MOVA, columns = ['Contrato', 'Anexo', 'Número de documento', 'Fecha Vencimiento',
       'Monto Capital', 'Monto intérés', 'Monto IVA', 'Tipo de Movimiento'])

## Adecuaciones

In [11]:
MOVA['Fecha Vencimiento'] = MOVA['Fecha Vencimiento'].fillna("2099-12-31")
print("Hay que revisar MOVA" if (MOVA.isna().any(0).sum() > 0) else "Todo bien con MOVA")
MOVA['Contrato'] = MOVA['Contrato'].astype(int)
MOVA_id = MOVA['Contrato']
print(MOVA.shape)
MOVA.head(1)

Todo bien con MOVA
(4518, 8)


,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210,2021-10-25,34000.0,8245.12,1319.22,FAC


In [12]:
MOVA = MOVA.loc[[mi in DEU_id for mi in MOVA_id]]
MOVA = MOVA.drop_duplicates('Contrato')
np.setdiff1d(MOVA['Contrato'], DEU_id)

array([], dtype=int32)

In [13]:
new_mova = []
for row in tqdm(MOVA.iterrows()):
    #new_mova = pd.concat([new_mova, row[1]], axis = 1)
    #new_mova.append(row[1])
    num = 16
    for i in range(1, num + 1):
        new_row = row[1].copy()
        new_row['Número de documento'] = str(new_row['Número de documento']) + "_"+str(i)
        new_row['Monto Capital'] = new_row['Monto Capital'] / num
        new_row['Monto intérés'] = new_row['Monto intérés'] / num
        new_row['Monto IVA'] = new_row['Monto IVA'] / num
        #new_mova = pd.concat([new_mova, new_row], axis = 1)
        new_mova.append(new_row)

2444it [00:08, 280.79it/s]


In [14]:
MOVA = pd.DataFrame(new_mova)
MOVA.head()

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210_1,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_2,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_3,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_4,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_5,2021-10-25,2125.0,515.32,82.45125,FAC


In [15]:
MOVA.to_csv(f'{path}/MOVA_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

# MOVIC

In [16]:
cursor = cnx.cursor()
sql_query = f"""
SELECT 
coalesce(cd.id_red, pd.id_distribuidor) as `Contrato`,
"N" AS `Anexo`,
-- coalesce(if(cd.tipo_prestamo = 'PRESTAMO PERSONAL RED', cd.id_red, cd.id_distribuidor), pd.id_distribuidor) as `Número de documento`,
coalesce(cd.id_distribuidor, pd.id_distribuidor) as `Número de documento`,
DATE(pd.fecha_pago) as `Fecha Depósito`,
DATE(pd.fecha_aplicacion_pago) as `Fecha Aplicación`,
IF(pd.seguro_asociado>0,-pd.seguro_asociado,pd.seguro_asociado) as `Monto de Seguro Asociado`,
IF(pd.seguro_distribuidor>0,-pd.seguro_distribuidor,pd.seguro_distribuidor) as `Monto de Seguro Distribuidor`,
"PAG" AS `Tipo de Movimiento`,
pd.procedencia as `Cuenta`,
pd.id_pago as `paymentId`

FROM pagos_desglosados pd

left JOIN (
		select 
			ca.id_distribuidor, max(ca.id_red) as 'id_red', max(ca.tipo_prestamo) as 'tipo_prestamo'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) cd on pd.id_distribuidor = cd.id_red
WHERE date(pd.fecha_pago) >='2021-01-01'
"""
cursor.execute(sql_query)
MOVIC = cursor.fetchall()

In [17]:
MOVIC = pd.DataFrame(MOVIC, columns = ['Contrato', 'Anexo', 'Número de documento', 'Fecha Depósito',
       'Fecha Aplicación', 'Monto de Seguro Asociado',
       'Monto de Seguro Distribuidor', 'Tipo de Movimiento', 'Cuenta',
       'paymentId'])

In [18]:
print("Hay que revisar MOVIC" if (MOVIC.isna().any(0).sum() > 0) else "Todo bien con MOVIC")
MOVIC.Contrato = MOVIC.Contrato.astype(int) 

MOVIC_id = MOVIC['Contrato']
print(MOVIC.shape)
MOVIC.head(1)

Todo bien con MOVIC
(337062, 10)


,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Seguro Asociado,Monto de Seguro Distribuidor,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-50,-25,PAG,BANCOMER,105234


In [19]:
sum([mi in DEU_id for mi in MOVIC_id])

256802

In [20]:
MOVIC = MOVIC.loc[[mi in DEU_id for mi in MOVIC_id]]

In [21]:
np.setdiff1d(MOVIC['Contrato'], DEU_id)

array([], dtype=int32)

In [22]:
MOVIC = MOVIC.drop_duplicates(subset = ["paymentId"])

In [23]:
MOVIC.to_csv(f'{path}/MOVIC_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

# MOVO

In [24]:
cursor = cnx.cursor()
sql_query = f"""
select
coalesce(cd.id_red, pd.id_distribuidor) as `Contrato`,
"N" AS `Anexo`,
-- coalesce(if(cd.tipo_prestamo = 'PRESTAMO PERSONAL RED', cd.id_red, cd.id_distribuidor), pd.id_distribuidor) as `Número de documento`,
coalesce(cd.id_distribuidor, pd.id_distribuidor) as `Número de documento`,
DATE(pd.fecha_pago) as `Fecha Depósito`,
DATE(pd.fecha_aplicacion_pago) as `Fecha Aplicación`,
IF(pd.capital>0,-pd.capital,pd.capital) as `Monto de Principal`,
IF(pd.interes>0,-pd.interes,pd.interes) as `Monto de interés`,
IF(pd.iva>0,-pd.iva,pd.iva) as `Monto de IVA`,
"1" AS `Tipo de Movimiento`,
pd.procedencia as `Cuenta`,
pd.id_pago as `paymentId`
FROM pagos_desglosados pd
left JOIN (
		select 
			ca.id_distribuidor, max(ca.id_red) as 'id_red', max(ca.tipo_prestamo) as 'tipo_prestamo'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) cd on pd.id_distribuidor = cd.id_red
where date(fecha_pago) >= '2021-01-01'
            """
cursor.execute(sql_query)
MOVO = cursor.fetchall()

In [25]:
MOVO = pd.DataFrame(MOVO, columns =  ['Contrato', 'Anexo', 'Número de documento', 'Fecha Depósito',
       'Fecha Aplicación', 'Monto de Principal', 'Monto de interés',
       'Monto de IVA', 'Tipo de Movimiento', 'Cuenta', 'paymentId'])

In [26]:
print("Hay que revisar MOVO" if (MOVO.isna().any(0).sum() > 0) else "Todo bien con MOVO")
MOVO['Contrato'] = MOVO['Contrato'].astype(int)
MOVO_id = MOVO['Contrato']
print(MOVO.shape)
MOVO.head(1)

Todo bien con MOVO
(337062, 11)


,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Principal,Monto de interés,Monto de IVA,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-1199.75,-103.78,-16.6,1,BANCOMER,105234


In [27]:
sum([mi in DEU_id for mi in MOVO_id])

256802

In [28]:
MOVO = MOVO.loc[[mi in DEU_id for mi in MOVO_id]]

In [29]:
np.setdiff1d(MOVO['Contrato'], DEU_id)

array([], dtype=int32)

In [30]:
MOVO = MOVO.drop_duplicates(subset = ["paymentId"])

In [31]:
MOVO.to_csv(f'./{path}/MOVO_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [32]:
cnx.close()

# Enviando al SFTP

In [33]:
import paramiko

# Configura la conexión SFTP
hostname = 'linqsy.dyndns.org'
port = 22
username = 'Volana'
password = '*v0lana-202311'

# Crea una instancia de la clase Transport
transport = paramiko.Transport((hostname, port))

# Conéctate con el usuario y contraseña
transport.connect(username=username, password=password)

sftp = paramiko.SFTPClient.from_transport(transport)

try:
    sftp.mkdir(f"archivos_{fecha_ayer_formato}")
    #sftp.mkdir(f"carga_inicial_2024_05_15")
    print(f'Directorio archivos_{fecha_ayer_formato} creado con éxito.')
except Exception as e:
    print(f'Error al crear el directorio: {e}')

"""
"""
# Cierra la conexión SFTP
sftp.close()

# Cierra la conexión de transporte
transport.close()


Directorio archivos_2024-05-15 creado con éxito.


In [34]:
# Configura la conexión SFTP
def enviar(name):
    hostname = 'linqsy.dyndns.org'
    port = 22
    username = 'Volana'
    password = '*v0lana-202311'

    # Crea una instancia de la clase Transport
    transport = paramiko.Transport((hostname, port))

    # Conéctate con el usuario y contraseña
    transport.connect(username=username, password=password)


    # Ruta local y remota del archivo
    local_path = f'./JERS/data/{fecha_ayer_formato}/{name}_{fecha_ayer_formato}.txt'
    #remote_path = f'/archivos_{fecha_ayer_formato}/{name}_{fecha_ayer_formato}.txt'
    #
    remote_path = f'/carga_inicial_2024_05_15/{name}_{fecha_ayer_formato}.txt'
    # Crea una instancia de la clase SFTP
    sftp = paramiko.SFTPClient.from_transport(transport)

    # Sube el archivo
    sftp.put(local_path, remote_path)

    # Cierra la conexión SFTP
    sftp.close()

    # Cierra la conexión de transporte
    transport.close()

In [35]:
for name in ['DEU', 'MOVA', 'MOVIC', 'MOVO']:
    enviar(name)
    print(f"{name} enviado correctamente")

PermissionError: [Errno 13] Permission denied